In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas tqdm sacrebleu

import os
import pandas as pd
from tqdm import tqdm
from sacrebleu.metrics import BLEU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 4.3 MB/s eta 0:00:00


In [ ]:
# Define constants
DIALECTS = ['EAAVE', 'IndE', 'JamE', 'CollSgE', 'ChcE']
DATASETS = ['WSC (659)', 'SST-2 (1000)', 'MultiRC (1000)', 'COPA (500)', 'BoolQ (1000)']
THRESHOLD = 0.7

# Dictionary mapping datasets to their column names
COLUMN_MAPPING = {
    'BoolQ (1000)': ('SAE Passage', 'SAE Passage'),
    'COPA (500)': ('Premise', 'Premise'),
    'MultiRC (1000)': ('Paragraph', 'Paragraph'),
    'SST-2 (1000)': ('Original Sentence', 'Original Sentence'),
    'WSC (659)': ('Original Paragraph', 'Original Paragraph')
}

def create_output_directories(dialect):
    base_path = f'/content/drive/MyDrive/!!Multi-AAVENUE/BLEU Score Filtered Datasets/{dialect}/GLUE + SuperGLUE'
    for dataset in DATASETS:
        os.makedirs(f'{base_path}/{dataset}', exist_ok=True)
    return base_path

def calculate_bleu_score(reference, candidate):
    try:
        bleu = BLEU(effective_order=True)
        return bleu.sentence_score(candidate, [reference]).score / 100
    except:
        return 0.0

def process_dataset(dialect, dataset, input_path, output_base_path):
    try:
        # Get column names
        sae_col, dialect_col_suffix = COLUMN_MAPPING[dataset]
        dialect_col = f'{dialect} ({sae_col})'
        bleu_col = f'BLEU Score {sae_col}'

        # Read dataset
        df = pd.read_csv(input_path)

        # Convert columns to string and handle NaN values
        df[sae_col] = df[sae_col].fillna('').astype(str)
        df[dialect_col] = df[dialect_col].fillna('').astype(str)

        # Calculate BLEU scores with progress bar
        tqdm.pandas(desc=f'Calculating BLEU scores for {dialect} - {dataset}')
        df[bleu_col] = df.progress_apply(
            lambda row: calculate_bleu_score(row[sae_col], row[dialect_col]),
            axis=1
        )

        # Save full dataset with BLEU scores
        output_path = f'{output_base_path}/{dataset}'
        df.to_csv(f'{output_path}/{dataset}_bleu_scores.csv', index=False)

        # Save filtered dataset
        filtered_df = df[df[bleu_col] < THRESHOLD]
        filtered_df.to_csv(f'{output_path}/{dataset}_filtered_bleu_scores.csv', index=False)

        # Calculate and save statistics
        total_samples = len(df)
        below_threshold = len(filtered_df)
        percentage = (below_threshold / total_samples) * 100

        with open(f'{output_path}/{dataset}_percentage.txt', 'w') as f:
            f.write(f'Total Samples: {total_samples}\n')
            f.write(f'Samples Below Threshold (<0.7): {below_threshold}\n')
            f.write(f'Percentage Below Threshold: {percentage:.2f}%')

        return total_samples, below_threshold, percentage

    except Exception as e:
        print(f'\nError in process_dataset for {dialect} - {dataset}: {str(e)}')
        return 0, 0, 0

def main():
    for dialect in tqdm(DIALECTS, desc='Processing dialects'):
        output_base_path = create_output_directories(dialect)

        for dataset in tqdm(DATASETS, desc=f'Processing datasets for {dialect}'):
            input_path = f'/content/drive/MyDrive/!!Multi-AAVENUE/GPT 4o + Multi-VALUE Translated Datasets/GPT 4o/{dialect}/Glue + SuperGlue/{dataset}_{dialect}.csv'

            try:
                total, below, percentage = process_dataset(
                    dialect,
                    dataset,
                    input_path,
                    output_base_path
                )
                if total > 0:
                    print(f'\nProcessed {dataset} for {dialect}:')
                    print(f'Total: {total}, Below threshold: {below}, Percentage: {percentage:.2f}%')
            except Exception as e:
                print(f'\nError processing {dataset} for {dialect}: {str(e)}')

if __name__ == '__main__':
    main()

Processing datasets for EAAVE:   0%|          | 0/5 [00:00<?, ?it/s]

Calculating BLEU scores for EAAVE - WSC (659):   0%|          | 0/658 [00:00<?, ?it/s]

Calculating BLEU scores for EAAVE - WSC (659):  25%|██▍       | 164/658 [00:00<00:00, 1632.75it/s]

Calculating BLEU scores for EAAVE - WSC (659):  57%|█████▋    | 374/658 [00:00<00:00, 1902.89it/s]

Calculating BLEU scores for EAAVE - WSC (659): 100%|██████████| 658/658 [00:00<00:00, 1705.60it/s]

Processing datasets for EAAVE:  20%|██        | 1/5 [00:00<00:01,  2.04it/s]


Processed WSC (659) for EAAVE:
Total: 658, Below threshold: 658, Percentage: 100.00%




Calculating BLEU scores for EAAVE - SST-2 (1000):   0%|          | 0/1000 [00:00<?, ?it/s]

Calculating BLEU scores for EAAVE - SST-2 (1000):  19%|█▉        | 188/1000 [00:00<00:00, 1829.95it/s]

Calculating BLEU scores for EAAVE - SST-2 (1000):  37%|███▋      | 371/1000 [00:00<00:00, 1525.69it/s]

Calculating BLEU scores for EAAVE - SST-2 (1000):  63%|██████▎   | 634/1000 [00:00<00:00, 1977.40it/s]

Calculating BLEU scores for EAAVE - SST-2 (1000): 100%|██████████| 1000/1000 [00:00<00:00, 2128.22it/s]

Processing datasets for EAAVE:  40%|████      | 2/5 [00:01<00:01,  1.87it/s]


Processed SST-2 (1000) for EAAVE:
Total: 1000, Below threshold: 1000, Percentage: 100.00%




Calculating BLEU scores for EAAVE - MultiRC (1000):   0%|          | 0/1000 [00:00<?, ?it/s]

Calculating BLEU scores for EAAVE - MultiRC (1000):   2%|▏         | 17/1000 [00:00<00:05, 166.92it/s]

Calculating BLEU scores for EAAVE - MultiRC (1000):   3%|▎         | 34/1000 [00:00<00:06, 146.61it/s]

Calculating BLEU scores for EAAVE - MultiRC (1000):   6%|▌         | 58/1000 [00:00<00:05, 183.87it/s]

Calculating BLEU scores for EAAVE - MultiRC (1000):   8%|▊         | 83/1000 [00:00<00:04, 208.45it/s]

Calculating BLEU scores for EAAVE - MultiRC (1000):  11%|█         | 106/1000 [00:00<00:04, 214.74it/s]

Calculating BLEU scores for EAAVE - MultiRC (1000):  13%|█▎        | 128/1000 [00:00<00:04, 215.08it/s]

Calculating BLEU scores for EAAVE - MultiRC (1000):  15%|█▌        | 154/1000 [00:00<00:03, 228.98it/s]

Calculating BLEU scores for EAAVE - MultiRC (1000):  18%|█▊        | 181/1000 [00:00<00:03, 241.61it/s]

Calculating BLEU scores for EAAVE - MultiRC (1000):  22%|██▏       |

...


Processed MultiRC (1000) for ChcE:
Total: 1000, Below threshold: 1000, Percentage: 100.00%




Calculating BLEU scores for ChcE - COPA (500):   0%|          | 0/500 [00:00<?, ?it/s]

Calculating BLEU scores for ChcE - COPA (500): 100%|██████████| 500/500 [00:00<00:00, 3578.86it/s]

Processing datasets for ChcE:  80%|████████  | 4/5 [00:06<00:01,  1.61s/it]


Processed COPA (500) for ChcE:
Total: 500, Below threshold: 481, Percentage: 96.20%




Calculating BLEU scores for ChcE - BoolQ (1000):   0%|          | 0/999 [00:00<?, ?it/s]

Calculating BLEU scores for ChcE - BoolQ (1000):   9%|▉         | 92/999 [00:00<00:00, 913.77it/s]

Calculating BLEU scores for ChcE - BoolQ (1000):  18%|█▊        | 184/999 [00:00<00:00, 865.73it/s]

Calculating BLEU scores for ChcE - BoolQ (1000):  27%|██▋       | 271/999 [00:00<00:00, 808.98it/s]

Calculating BLEU scores for ChcE - BoolQ (1000):  35%|███▌      | 353/999 [00:00<00:00, 773.29it/s]

Calculating BLEU scores for ChcE - BoolQ (1000):  43%|████▎     | 431/999 [00:00<00:00, 743.26it/s]

Calculating BLEU scores for ChcE - BoolQ (1000):  51%|█████     | 506/999 [00:00<00:00, 682.23it/s]

Calculating BLEU scores for ChcE - BoolQ (1000):  58%|█████▊    | 575/999 [00:00<00:00, 673.35it/s]

Calculating BLEU scores for ChcE - BoolQ (1000):  65%|██████▍   | 646/999 [00:00<00:00, 682.57it/s]

Calculating BLEU scores for ChcE - BoolQ (1000):  72%|███████▏  | 715/999 [00:01<00:00, 658.51it/s]




Processed BoolQ (1000) for ChcE:
Total: 999, Below threshold: 994, Percentage: 99.50%
